# Extraindo dados da Covid19 via API
### Fonte de Dados: https://brasil.io/dataset/covid19/
### Augusto SPINELLI

### Licença
Os dados dados convertidos estão sob a licença Creative Commons Attribution ShareAlike. Caso utilize os dados, cite a fonte original e quem tratou os dados, como: Fonte: Secretarias de Saúde das Unidades Federativas, dados tratados por Álvaro Justen e colaboradores/Brasil.IO. Caso compartilhe os dados, utilize a mesma licença.

#### Para criação do nosso modelo preditivo que tentará prever se um paciente covid irá a óbito ou não, utilizaremos o dataset de casos individualizados anonimos baseado no cep do paciente

## Data Munging

É o processo de mapeamento e transformação de dados para análise. No nosso caso, temos acesso somente a um dataset que informa a evolução do paciente com base no endereço / CEP de residencia, e tentaremos preparar um modelo que irá prever a evolucão de pacientes ativos de acordo com as áreas de planejamento Cidade do Rio de Janeiro (APS)


In [30]:
#importando dependencias
import pandas as pd
import numpy as np
import os

#Caminho dos arquivos RAW
raw_data_path = os.path.join(os.path.pardir, 'data', 'raw')
data_path = os.path.join(raw_data_path, 'ceps2.csv')

#criando dataframes brutos/raw
raw_df = pd.read_csv(data_path)

#verificando informações do dataframe
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50640 entries, 0 to 50639
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   dt_notific                 50613 non-null  object 
 1   dt_inicio_sintomas         50603 non-null  object 
 2   bairro_resid__estadia      50640 non-null  object 
 3   ap_residencia_estadia      50640 non-null  float64
 4   evolu��o                   50640 non-null  object 
 5   dt_�bito                   4727 non-null   object 
 6   CEP                        50640 non-null  int64  
 7   Data_atualiza��o;OBJECTID  50640 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 3.1+ MB


In [31]:
#renomeando colunas com encoding errado
df = raw_df.rename(columns={"evolu��o": "evolucao", "dt_�bito": "dt_obito"})

In [32]:
#mapeando data de atualizacao de forma correta
def GetData(dataFull):
    d = dataFull.split(';')[0]
    return d

df['dt_registro'] = df['Data_atualiza��o;OBJECTID'].map(lambda x: GetData(x))

In [33]:
#removendo coluna inútil
del df['Data_atualiza��o;OBJECTID']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50640 entries, 0 to 50639
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dt_notific             50613 non-null  object 
 1   dt_inicio_sintomas     50603 non-null  object 
 2   bairro_resid__estadia  50640 non-null  object 
 3   ap_residencia_estadia  50640 non-null  float64
 4   evolucao               50640 non-null  object 
 5   dt_obito               4727 non-null   object 
 6   CEP                    50640 non-null  int64  
 7   dt_registro            50640 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 3.1+ MB


In [34]:
recuperados = df.loc[df.evolucao == 'recuperado',:] 
print('Nuúmero de Recuperados: {0}'.format(len(recuperados)))

ativos = df.loc[df.evolucao == 'ativo',:] 
print('Nuúmero de Ativos: {0}'.format(len(ativos)))

obitos = df.loc[df.evolucao == '�bito',:] 
print('Nuúmero de Óbitos: {0}'.format(len(obitos)))

Nuúmero de Recuperados: 44723
Nuúmero de Ativos: 1190
Nuúmero de Óbitos: 4727


In [35]:
df.ap_residencia_estadia.unique()

array([3.2, 3.3, 2.2, 4. , 3.1, 5.1, 5.2, 1. , 2.1, 5.3])

In [36]:
df = pd.get_dummies(df,columns=['ap_residencia_estadia'])

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50640 entries, 0 to 50639
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   dt_notific                 50613 non-null  object
 1   dt_inicio_sintomas         50603 non-null  object
 2   bairro_resid__estadia      50640 non-null  object
 3   evolucao                   50640 non-null  object
 4   dt_obito                   4727 non-null   object
 5   CEP                        50640 non-null  int64 
 6   dt_registro                50640 non-null  object
 7   ap_residencia_estadia_1.0  50640 non-null  uint8 
 8   ap_residencia_estadia_2.1  50640 non-null  uint8 
 9   ap_residencia_estadia_2.2  50640 non-null  uint8 
 10  ap_residencia_estadia_3.1  50640 non-null  uint8 
 11  ap_residencia_estadia_3.2  50640 non-null  uint8 
 12  ap_residencia_estadia_3.3  50640 non-null  uint8 
 13  ap_residencia_estadia_4.0  50640 non-null  uint8 
 14  ap_res

In [38]:
df.drop(['dt_obito','dt_notific','dt_inicio_sintomas','bairro_resid__estadia','CEP','dt_registro'], axis=1,inplace=True)

In [39]:
def preencheSobreviveu(evolucao):
    ret = -1
    if evolucao == 'recuperado':
        ret = 1
    elif evolucao == 'ativo':
        ret = -1 #não sabemos - test data
    else:
        ret = 0 #óbito
    return ret
    
df['sobreviveu'] = df['evolucao'].map(lambda x: preencheSobreviveu(x))

In [40]:
df.drop(['evolucao'], axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50640 entries, 0 to 50639
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   ap_residencia_estadia_1.0  50640 non-null  uint8
 1   ap_residencia_estadia_2.1  50640 non-null  uint8
 2   ap_residencia_estadia_2.2  50640 non-null  uint8
 3   ap_residencia_estadia_3.1  50640 non-null  uint8
 4   ap_residencia_estadia_3.2  50640 non-null  uint8
 5   ap_residencia_estadia_3.3  50640 non-null  uint8
 6   ap_residencia_estadia_4.0  50640 non-null  uint8
 7   ap_residencia_estadia_5.1  50640 non-null  uint8
 8   ap_residencia_estadia_5.2  50640 non-null  uint8
 9   ap_residencia_estadia_5.3  50640 non-null  uint8
 10  sobreviveu                 50640 non-null  int64
dtypes: int64(1), uint8(10)
memory usage: 890.3 KB


In [41]:
#reorder columns
columns = [column for column in df.columns if column != 'sobreviveu']
columns = ['sobreviveu'] + columns
df = df[columns]

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50640 entries, 0 to 50639
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   sobreviveu                 50640 non-null  int64
 1   ap_residencia_estadia_1.0  50640 non-null  uint8
 2   ap_residencia_estadia_2.1  50640 non-null  uint8
 3   ap_residencia_estadia_2.2  50640 non-null  uint8
 4   ap_residencia_estadia_3.1  50640 non-null  uint8
 5   ap_residencia_estadia_3.2  50640 non-null  uint8
 6   ap_residencia_estadia_3.3  50640 non-null  uint8
 7   ap_residencia_estadia_4.0  50640 non-null  uint8
 8   ap_residencia_estadia_5.1  50640 non-null  uint8
 9   ap_residencia_estadia_5.2  50640 non-null  uint8
 10  ap_residencia_estadia_5.3  50640 non-null  uint8
dtypes: int64(1), uint8(10)
memory usage: 890.3 KB


## Salvando dados em arquivo

Nosso pobre modelo por hora considera como critério apenas a AP de residencia do paciente para definir se ele sobrevive ou não. Devemos procurar um outro dataset com dados anonimos e individualizados mais rica, como por exemplo sexo, renda, etc para que esses critérios também sejam considerados pelo nosso modelo na hora de prever se o paciente sobreviverá ou não.

Por hora, com o que temos em mãos, vamos salvar os datasets de test e training 

In [44]:
processed_data_path = os.path.join(os.path.pardir,'data','processed')
write_train_path = os.path.join(processed_data_path,'train.csv')
write_test_path = os.path.join(processed_data_path,'test.csv')

In [45]:
df.loc[df.sobreviveu != -1].to_csv(write_train_path)
columns = [column for column in df.columns if column != 'sobreviveu']
df.loc[df.sobreviveu == -1, columns].to_csv(write_test_path)

### End of Notebook